# Revisión de Flujos de potencia de elementos en casos de PSS/E

In [ ]:
import os
import sys

#Damos de alta los directorios de psse,para correrlo sin abrirlo

PSSPY_location = r'C:\Program Files\PTI\PSSE35\35.3\PSSPY39' #hay que cambiar la ruta dependianto si se usa psse34 o 35 tambien en base al python 2 o 3
PSSE_location = r'C:\Program Files\PTI\PSSE35\35.3\PSSBIN' #hay que cambiar la ruta dependianto si se usa psse34 o 35
sys.path.append(PSSPY_location)
sys.path.append(PSSE_location)
os.environ['PATH'] += ';' + PSSPY_location
os.environ['PATH'] += ';' + PSSE_location

#Importamos librerias del PSS

import psse35
import psspy
import redirect

#Importamos librería de PEMs

from src.PEMS import *

redirect.psse2py()
psspy.psseinit(50000)

In [ ]:
CASOS_PATH = "./casos/2023_20Hinvierno/" # Ruta donde se guardan los archivos .sav (de acuerdo con la estructura del repositorio)
casos_sav = os.listdir(CASOS_PATH) # Genera una lista con los nombres de 

In [ ]:
LIN_NOM = ['RAP - DMD',
           'RAP - SLR']
LINEAS = ['640004-5D-640099',
          '640004-G1-32277']

In [ ]:
def dic_flujos():
    sid = 0
    ierr = psspy.bsys(sid,
                  1,[115.0,400.0],             #kv    filter
                  0,[],                        #area  filter
                  0,[],                        #bus   filter
                  0,[],                        #owner filter
                  0,[])                        #zone  filter
 
    ierr, ids = psspy.abrnchar(0,1,3,4,2, string=["ID"])
    ierr, numeros = psspy.abrnint(0,1,3,4,2, string=["FROMNUMBER","TONUMBER"])
    ierr, flujo = psspy.afloereal(0,1,3,4,4, string=["P","Q","MVA","PLOSS"])
    #Crea claves de enlaces [#FROM_BUS - ID - #TO_BUS, #FROM_BUS - ID - #TO_BUS, ... ]
    ramas_list = []
    for elem in range(len(ids[0])):
        ramas_list.append(str(numeros[0][elem]) + '-' + str(ids[0][elem]) + '-' + str(numeros[1][elem]))

    valores = zip(*flujo)

    #Crea diccionario, donde la llave es la clave del enlace y sus valores son los flujos de potencia por el elemento:
    flujos_dict = {}
    for i in range(len(flujo)):
        flujos_dict[ramas_list[i]] = valores[i]
    return flujos_dict

In [ ]:

#Busca los valores de los enlaces deseados
f = []
for linea in LINEAS:
    f.append(flujos_dict[linea])


In [ ]:
for caso in casos_sav:
    psspy.case(os.path.abspath("casos/2023_20Hinvierno" + caso))
    